In [2]:
oldpath = '../Human_anno'
newpath = '../Human_anno_sora_test'

import os
import json

for filename in os.listdir(newpath):
    if filename.endswith('.json'):
        with open(os.path.join(newpath, filename), 'r') as f:
            new = json.load(f)
        with open(os.path.join(oldpath, filename), 'r') as f:
            old = json.load(f)

In [3]:
old

[{'prompt_en': 'Close up of grapes on a rotating table.',
  'videos': {'cogvideox5b': 'overall_consistency/cogvideox5b/Close up of grapes on a rotating table._0.mp4',
   'kling': 'overall_consistency/kling/Close up of grapes on a rotating table._0.mp4',
   'gen3': 'overall_consistency/gen3/Close up of grapes on a rotating table._0.mp4',
   'videocrafter2': 'overall_consistency/videocrafter2/Close up of grapes on a rotating table._0.mp4',
   'pika': 'overall_consistency/pika/Close up of grapes on a rotating table._0.mp4',
   'show1': 'overall_consistency/show1/Close up of grapes on a rotating table._0.mp4',
   'lavie': 'overall_consistency/lavie/Close up of grapes on a rotating table._0.mp4'},
  'gpt4o_eval': {'cogvideox5b': 4,
   'kling': 3,
   'gen3': 5,
   'videocrafter2': 4,
   'pika': 2,
   'show1': 4,
   'lavie': 3},
  'human_anno': {'cogvideox5b': [4, 3, 5, 2],
   'kling': [4, 3, 2, 1],
   'gen3': [5, 4, 4, 3],
   'videocrafter2': [4, 3, 5, 4],
   'pika': [2, 2, 4, 2],
   'show1'

In [5]:
new

[{'prompt_en': 'Close up of grapes on a rotating table.',
  'videos': {'cogvideox5b': 'overall_consistency/cogvideox5b/Close up of grapes on a rotating table._0.mp4',
   'gen3': 'overall_consistency/gen3/Close up of grapes on a rotating table._0.mp4',
   'kling': 'overall_consistency/kling/Close up of grapes on a rotating table._0.mp4',
   'videocrafter2': 'overall_consistency/videocrafter2/Close up of grapes on a rotating table._0.mp4',
   'pika': 'overall_consistency/pika/Close up of grapes on a rotating table._0.mp4',
   'show1': 'overall_consistency/show1/Close up of grapes on a rotating table._0.mp4',
   'lavie': 'overall_consistency/lavie/Close up of grapes on a rotating table._0.mp4',
   'sora': 'overall_consistency/sora/Close up of grapes on a rotating table._0.mp4'},
  'gpt4o_eval': {'cogvideox5b': 0,
   'gen3': 0,
   'kling': 0,
   'videocrafter2': 0,
   'pika': 0,
   'show1': 0,
   'lavie': 0,
   'sora': 0},
  'human_anno': {'cogvideox5b': [],
   'gen3': [],
   'kling': [],
